In [18]:
import pandas as pd


# Read the TSV file into a Pandas DataFrame
df1 = pd.read_csv('../data/raw/filtered.tsv', sep='\t', index_col=0)

# Display the DataFrame
df1.head(5)


,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348


In [19]:
df2 = pd.read_csv('../data/external/paradetox.tsv', sep='\t', index_col=0)

# Display the DataFrame
df2.head(5)

,neutral1,neutral2,neutral3
toxic,,,
he had steel balls too !,he was brave too!,NaN,NaN
"dude should have been taken to api , he would be right at home with all the other knuckleheads there",It would have been good if he went to api. He ...,NaN,NaN
"im not gonna sell the fucking picture , i just want to contribute to the fucking article .","I'm not gonna sell the picture, i just want to...",NaN,NaN
the garbage that is being created by cnn and other news agencies is outrageous .,the news that is being created by cnn and othe...,The news that is being created by cnn and othe...,the garbage that is being created by cnn and o...
the reason they dont exist is because neither is a pathological liar like trump .,The reason they don't exist is because neither...,NaN,NaN
